In [101]:
import pandas as pd
import numpy as np  
import glob  
import sys
import warnings
import os
import time

In [102]:
df = pd.read_pickle('../data/pkl/merged_filtered_0620.pkl')

: 

In [100]:
len(df)

3558589

In [3]:
cat_lst = pd.read_excel('../data/decline_cities_kor_list.xlsx')
cat_lst = cat_lst[['name_kor', 'type']]
cat_lst


,name_kor,type
0,서울특별시,0 Recent decline
1,서울특별시 종로구,0 Continuous decline
2,서울특별시 중구,1 Recovery
3,서울특별시 용산구,1 Recovery
4,서울특별시 성동구,0 Continuous decline
...,...,...
230,경상남도 함양군,0 Continuous decline
231,경상남도 거창군,0 Continuous decline/slowdown
232,경상남도 합천군,0 Continuous decline
233,제주특별자치도 제주시,2 Continuous growth


In [4]:
cat_lst['name_kor'] = cat_lst['name_kor'].apply(lambda x : x.split())

In [5]:
lst = []
for item  in cat_lst['name_kor']:
    try:
        lst.append(item[1])
    except:
        lst.append('pass')

In [6]:
cat_lst['city'] = lst

In [7]:
import re

idx = []
for item in cat_lst['city']:
    idx.append(re.search('시$', item) != None)

In [8]:
cat_lst_use = cat_lst[idx]
cat_lst_use[['city', 'type']]

,city,type
81,동해시,0 Continuous decline
82,태백시,0 Continuous decline
83,삼척시,0 Continuous decline
84,공주시,0 Continuous decline
85,익산시,0 Continuous decline
...,...,...
220,진주시,2 Continuous growth
221,김해시,2 Continuous growth
222,양산시,2 Continuous growth
233,제주시,2 Continuous growth


In [9]:
df_group = pd.merge(left = df, right = cat_lst_use, how = 'left', on = 'city')

In [10]:
group = df_group.groupby('type')
group = [group.get_group(x) for x in group.groups]


In [11]:
key_lst = []

for df in group:
    col = df.columns
    key = df['키워드']
    keywords = ""
    features = ""
    for item in key:
        try:
            keywords += item.replace("," , " ")
        except:
            pass
    
    key_lst.append(keywords)

In [17]:
print(len(key_lst))

5


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
T_Vec = TfidfVectorizer(min_df= 5, analyzer='word', sublinear_tf= True)
DTM = T_Vec.fit_transform(key_lst)
tfidf = pd.DataFrame(DTM.toarray(), columns = T_Vec.get_feature_names())

/home/mglee/.conda/envs/svmglee/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [47]:
idx_lst = []
for type in group:
    idx_lst.append(str(type['type'].unique()))

In [48]:
idx_lst

["['0 Continuous decline']",
 "['0 Recent decline']",
 "['0 Recent decline/Peak']",
 "['1 Recovery']",
 "['2 Continuous growth']"]

In [49]:
tfidf = tfidf.T

In [50]:
tfidf.columns = idx_lst

In [79]:
tfidf.to_csv("../result/TF_IDF_by_group.csv")

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

C_Vec = CountVectorizer(min_df= 5, analyzer='word')
C_DTM = C_Vec.fit_transform(key_lst)
tf = pd.DataFrame(C_DTM.toarray(), columns = C_Vec.get_feature_names())

tf = tf.T
tf_sum = tf.apply(lambda x : sum(x))
tf_ratio = tf / tf_sum

common_term = tf_ratio.sum(axis = 1).sort_values(ascending= False)[:500]

/home/mglee/.conda/envs/svmglee/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [99]:
pd.Series(common_term.index).to_csv("../result/common_term.csv")

In [94]:
tfidf_by_group_df = pd.DataFrame()
for i in range(5):
    temp = tfidf.iloc[:,i].sort_values(ascending=False)
    temp = temp[:1000]
    temp = temp[set(temp.index) - set(common_term.index)]
    temp.sort_values(inplace = True, ascending= False)
    temp = temp.reset_index()
    print(len(temp))
    tfidf_by_group_df = pd.concat([tfidf_by_group_df, temp], axis = 1)

/tmp/ipykernel_14978/1275396579.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  temp = temp[set(temp.index) - set(common_term.index)]
/tmp/ipykernel_14978/1275396579.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  temp = temp[set(temp.index) - set(common_term.index)]


523
515


/tmp/ipykernel_14978/1275396579.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  temp = temp[set(temp.index) - set(common_term.index)]


521
526
521


/tmp/ipykernel_14978/1275396579.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  temp = temp[set(temp.index) - set(common_term.index)]
/tmp/ipykernel_14978/1275396579.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  temp = temp[set(temp.index) - set(common_term.index)]


In [95]:
tfidf_by_group_df

,index,['0 Continuous decline'],index,['0 Recent decline'],index,['0 Recent decline/Peak'],index,['1 Recovery'],index,['2 Continuous growth']
0,익산,0.004569,속초시,0.004555,부천,0.004629,제천,0.004528,제주시,0.003593
1,김제시,0.004553,과천시,0.004525,안산,0.004602,보령시,0.004476,천안시,0.003591
2,삼척,0.004548,목포시,0.004469,거제시,0.004581,나주시,0.004401,청주,0.003582
3,정읍시,0.004528,마산,0.004468,군포시,0.004530,광양시,0.004376,서귀포시,0.003530
4,공주,0.004519,속초,0.004467,판교,0.004497,논산시,0.004370,천안,0.003525
...,...,...,...,...,...,...,...,...,...,...
521,클러스터,0.003852,NaN,NaN,NaN,NaN,마당,0.003815,NaN,NaN
522,글로벌,0.003852,NaN,NaN,NaN,NaN,충청,0.003815,NaN,NaN
523,NaN,NaN,NaN,NaN,NaN,NaN,공정,0.003815,NaN,NaN
524,NaN,NaN,NaN,NaN,NaN,NaN,명소,0.003815,NaN,NaN


In [96]:
tfidf_by_group_df.to_csv("../result/TF_IDF_by_group_anti_common.csv")